In [4]:
import tkinter as tk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification 
import xgboost as xgb
import spacy
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

def initFunc(INPUT):
    #READ NORMALIZED DATA FOR TARGET VERSION OF PIPELINE
    df = pd.read_csv("normalizedResult.tsv", sep="\t")

    nlp = spacy.load("de_core_news_sm")
    tweet_words = []
    n_text = ""
    doc = nlp(INPUT)
    for token in doc:
        if token.is_alpha:
            n_text+=token.text.lower()+" "

    INPUT = n_text

    #CREATE TWO LISTS THAT CONTAIN EACH TEXT OF A TWEET AND ITS TARGET
    df=df.dropna()
    target=(df.iloc[0:8132 , 2 ].to_list())
    text=df.iloc[0:8132, 1 ].to_list()

    #CREATE DATAFRAME THAT CONTAINS TWEET TEXT AND TARGET
    df=pd.DataFrame ({'Text': text , 'Target': target })
    testdf=pd.DataFrame ({'Text': [INPUT] })

    X_train = df.Text
    y_train = df.Target
    X_test = testdf.Text
        
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    
    Xtrain = loadtxt('encodedModel.csv', delimiter=',')
    
    xt = X_test.tolist()
    Xtest = model.encode(xt)

    #create classifier and train model
    clf = xgb.XGBClassifier(objective="binary:logistic", random_state=2021,
                               n_estimators=100,
                               gamma=0,
                               min_child_weight=1,
                               learning_rate=0.3,
                               reg_alpha=6,
                               max_depth=6,
                               colsample_bytree=0.7, colsample_bylevel=0.6, colsample_bynode=1
                           )

    #2. fit with X_train and y_train
    clf.fit(Xtrain, y_train)

    #3. get the predictions for X_test and store it in y_pred
    y_pred = clf.predict(Xtest)
    
    return y_pred[0]
    
def button_click():
    output_label.config(text="")
    target = initFunc(input_field.get())
    if target == 0:
        output_label.config(text="Target: PERSON")
    if target == 1:
        output_label.config(text="Target: GROUP")
    if target == 2:
        output_label.config(text="Target: PUBLIC")

root = tk.Tk()
root.geometry("800x600")
root.title("Target Detection Demo")

header_label = tk.Label(root, text="Target Detection Demo", font=("Arial", 18), anchor="center")
header_label.pack(pady=(20,0))

input_frame = tk.Frame(root)
input_frame.pack(pady=(100,0), anchor="center")

input_field = tk.Entry(input_frame, font=("Arial", 18), width=40)
input_field.pack(side="left", pady=(20,20))

button = tk.Button(input_frame, text="Submit", font=("Arial", 14), command=button_click)
button.pack(side="left", padx=(20,0))

output_label = tk.Label(root, text="", font=("Arial", 14), anchor="s")
output_label.pack(pady=(200,0), anchor="center")

root.mainloop()